
# Entrenamiento y evaluación de un detector automatico de cantos basado en CNN con OpenSoundscape

### *Santiago Ruiz Guzman - Kitzes Lab, University of Pittsburgh*


Este tutorial lo guiara durante el entrenamiento y evaluacion de un clasificador con redes neuronales convolucionales (CNNs) para identificar vocalizaciones utilizando el paquete OpenSoundscape python.

Hay más tutoriales disponibles en el sitio web OpenSoundscape relacionados con el analisis de audio y espectrogramas o el uso de otro algoritmo de procesamiento de señales para detectar vocalizaciones.

 http://opensoundscape.org/en/latest/


Primero necesitamos instalar opensoundscape y sus dependencias. Siga el proceso de instalacion en la documentacion segun sea el caso https://opensoundscape.org/en/latest/installation/mac_and_linux.html


A continuación, cambie la ruta de la carpeta donde se encuentran los datos de la practica

In [ ]:
folder='/Users/santiagoruiz/Documents/AI_Andes/Sesiones_practicas/Segunda_practica/Entrenamiento_CNN'

### Paquetes y funciones necesarias

In [ ]:
import opensoundscape
from opensoundscape import CNN
import opensoundscape.ml
from opensoundscape.preprocess.preprocessors import SpectrogramPreprocessor
from opensoundscape.ml.datasets import AudioSplittingDataset
from opensoundscape import Audio, Spectrogram
from opensoundscape.preprocess.actions import Action
from opensoundscape.data_selection import resample
from opensoundscape.ml.cnn import load_model
from scipy import signal
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import random
import subprocess
from glob import glob
import sklearn
import os
from matplotlib import pyplot as plt
from IPython.display import display, HTML
plt.rcParams['figure.figsize']=[15,5]
%config InlineBackend.figure_format = 'retina'

# Configurar 'seed' aleatoria para garantizar reproducibilidad
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

Ahora vamos a definir algunas funciones que utilizaremos en el taller.

In [ ]:
# Funcion para leer los archivos WAV de entrenamiento
def read_samples(path,label):
    list_pos = []
    for filename in os.scandir(path):
        if filename.is_file():
            absolute = os.path.abspath(filename.path)
            list_pos.append(absolute)
    pos_df = pd.DataFrame(list_pos, columns=['file_name'])
    index_pos = pd.DataFrame(index=pos_df['file_name'])
    index_pos['boana'] = label
    return index_pos

# Graficar curvas de perdida
def plot_scatter(trained_model, title):
    plt.figure(figsize=(8, 2))
    plt.scatter(trained_model.loss_hist.keys(),trained_model.loss_hist.values(),linewidth=2,edgecolors='w')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.tight_layout()
    plt.title(title)
    plt.show()

# Graficar histogramas con puntajes de deteccion por clase
def plot_testing_hist (model_output,testing_df,title):
    first_column_list = model_output.iloc[:, 0].tolist()
    testing_df['scores']=first_column_list
    boana_pos = testing_df[testing_df['labels'] == True]
    boana_neg = testing_df[testing_df['labels'] == False]
    plt.figure(figsize=(10, 2))
    plt.hist(boana_pos['scores'], bins=20, alpha=0.5, label='True')
    plt.hist(boana_neg['scores'], bins=20, alpha=0.5, label='False')
    plt.xlabel('Detection score')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend(loc='upper right')
    plt.show()


# Entrenar un clasificador basado en CNN para *Boana faber*

# Como puedo representar un archivo de audio?

Durante el entrenamiento de un clasificador de cantos se utilizan archivos de sonido en diferentes formatos. A continuación encontrará varias funciones para cargar los cantos de su conjunto de datos directamente desde python (Compruebe https://opensoundscape.org/en/latest/tutorials/audio.html#Load-audio-files)



In [ ]:
# Con OpenSoundscape tu puedes cargar archivos WAV como un objeto de audio
audio_path= folder + '/Testing/1_pos_INCT20955_20191011_214500_28_31.wav'
audio_object = Audio.from_file(audio_path)

# Dale click al boton de reproducir para escuchar los audios
audio_object

In [ ]:
# Verifiquemos los metadatos de los audios
audio_object.metadata

In [ ]:
# Grafica el oscilograma
plt.plot(audio_object.samples)
plt.show()

In [ ]:
# Grafica el espectrograma
spectrogram_object = Spectrogram.from_audio(audio_object)
spectrogram_object.plot()

## Construyamos el set de entrenamiento

En este ejercicio vamos a implementar un clasificador binario. Para ello es necesario crear un conjunto de entrenamiento con muestras de audio de las clases de interés.

OpenSoundscape también permite construir un conjunto de entrenamiento a partir de archivos de anotación de Raven, esta forma se recomienda con el fin de evitar la creación de archivos duplicados (https://opensoundscape.org/en/latest/tutorials/annotations.html#Load-multiple-Raven-annotation-tables).


In [ ]:
# Carpeta donde las grabaciones de la primera clase estan guardadas
positive=folder+'/Training/BOAFAB_Baseline'

list_pos=[] # Crea una lista vacia
for filename in os.scandir(positive):   # Revisa todas las rutas al interior de la carpeta donde los WAVs estan guardados
    absolute=os.path.abspath(filename)  # Obten las rutas absoluta de cada grabacion
    list_pos.append(absolute)           # Guarda todas las rutas en una lista

# Crea una dataframe con la lista
pos_df = pd.DataFrame(list_pos,columns=['file_name'])

# Crear un nuevo dataframe con los nombres de archivo de la tabla anterior como el índice, para este modelo, las rutas wavs NECESITA ser el índice, no sólo una columna normal
index_pos = pd.DataFrame(index=pos_df['file_name'])
index_pos['boana'] = "1" # Crea una columna con 1 representando la presencia de la especie objetivo
index_pos

In [ ]:
# Carpeta donde las grabaciones negativas estan guardadas
negatives_path=folder+'/Training/OTHER'
index_neg=read_samples(negatives_path,"0")

dataset_size=200 #@param
index_pos=index_pos.head(dataset_size)
index_neg=index_neg.head(dataset_size)

frames = [index_pos, index_neg]
label = pd.concat(frames)

label["boana"] = pd.to_numeric(label["boana"])
label


## Parte el set de datos entre el set de entrenamiento y validacion



Teniendo en cuenta los múltiples parámetros que podemos ajustar en un clasificador, es importante establecer un subconjunto de llamadas anotadas para ayudar al modelo a saber lo bueno que es para hacer clasificaciones precisas. Este subconjunto de llamadas anotadas (diferentes de las del conjunto de entrenamiento) se denomina conjunto de validación. Dependiendo del número de llamadas anotadas disponibles, el conjunto de validación suele estar formado por el 20% de las cantos por clase.

In [ ]:
train_df, valid_df = train_test_split(label, # Dataframe con las rutas para ambas clases
                                      test_size=0.2, # Proporción de grabaciones asignadas en el conjunto de validación (20% en este caso)
                                      random_state=0) # Número fijo significa que la división «aleatoria» será exactamente la misma cada vez que la ejecutemos

print(f"Set de entrenamiento con {len(train_df)} cantos  y set de validacion con {len(valid_df)} cantos")
print(f"Set de entrenamiento con  {len(train_df)} cantos  y set de validacion con {len(valid_df)} cantos")

## Selecciona una arquitectura para un clasificador

Elegir la arquitectura adecuada (número, tipos y dimensiones de las capas dentro de la CNN) para una CNN es un paso importante en el desarrollo de un clasificador acústico. Aunque es posible construir un clasificador desde cero (estableciendo distintas combinaciones de capas), la forma más habitual de construir clasificadores acústicos es utilizando una red preentrenada, proceso conocido como aprendizaje por transferencia.

En este caso, tanto la arquitectura como los pesos del clasificador se basarán en la ResNET18, una popular red preentrenada entrenada en ImageNET.

In [ ]:
classes = label.columns
architecture= 'resnet18' #@param
baseline = CNN(architecture,     # Red pre entrenada
            classes,             # El numero de clases
            sample_duration=3.0) # Duracion en segundos de cada grabacion

# Miremos la arquitectura de la ResNET18
baseline

In [ ]:
# Pretrained networks available in OPSO
opensoundscape.ml.cnn_architectures.list_architectures()

# Conoce al preprocesador

En este punto, ya hemos seleccionado el conjunto de datos para los set de entrenamiento y validación, así como la arquitectura para nuestro clasificador. Sin embargo, como puede verse a continuación, cada una de las llamadas de nuestro conjunto de datos debe transformarse en otras estructuras de datos para poder ser interpretada por la CNN. Empezando por un archivo .wav, cada llamada se convertirá en un espectrograma y, finalmente, en un tensor, que es básicamente un vector que contiene la información del espectrograma.

Mientras cada archivo .wav pasa por este proceso, es posible establecer múltiples características para mejorar la generalización del modelo. Esto comienza con cambios en la configuración del espectrograma e incluye la creación de grabaciones «falsas» con ligeras modificaciones, lo que se conoce como aumento de datos.

Hablaremos de ello más adelante. Por ahora, permítame presentarle la forma en que OpenSoundscape visualiza el preprocesador:

In [ ]:
# OPSO incluye alguna de las acciones de aumentacion de datos, ignoremolas por ahora
baseline.preprocessor.pipeline.add_noise.bypass=True
baseline.preprocessor.pipeline.time_mask.bypass=True
baseline.preprocessor.pipeline.bandpass.bypass=True
baseline.preprocessor.pipeline.frequency_mask.bypass=True
baseline.preprocessor.pipeline.random_affine.bypass=True
baseline.preprocessor # Despliega el preprocesador

# Introduzcamos el set de evaluacion

Un buen conjunto de evaluación es clave para el desarrollo de cualquier clasificador automático. Debe incluir cantos nuevos (no presentes en el conjunto de entrenamiento o evaluación) de todas las clases, a ser posible con la mayor variación posible entre ellas.

Además de las métricas tradicionales, la comparación entre las puntuaciones de detección del conjunto de evaluación es muy útil para conocer el grado de generalización entre múltiples modelos.

In [ ]:
# Lista los archivos WAV en la carpeta de evaluacion
testing_list=sorted(glob(folder + '/Testing/*.wav'))

len(testing_list)

In [ ]:

# Columnas con las etiquetas del set de evaluacion
testing_labels = [True] * 100 + [False] * 100
# Dataframe final
testing_df = pd.DataFrame({'paths': testing_list, 'labels': testing_labels})
testing_df

# Entrena (y evalua) el modelo


Con el preprocesador listo, es hora del paso más crucial del proceso: entrenar el modelo. En este paso, llamaremos tanto al conjunto de entrenamiento como al conjunto de validación que hemos creado. Definiremos una ruta en la que se almacenarán todos los modelos (con extensión .model) y tendremos que definir hiperparámetros importantes como el número de épocas y el tamaño del lote.

Este es el paso del proceso que más tiempo consume. Dependiendo del tamaño del conjunto de datos, la complejidad de la arquitectura de la CNN y los valores de los hiperparámetros (especialmente el número de épocas), puede llevar horas. Por eso es importante hacer un seguimiento de los ajustes que utilizamos en cada sesión de entrenamiento. Para ello, podemos utilizar programas como Weights and Biases (https://opensoundscape.org/en/latest/tutorials/train_cnn.html#Set-up-WandB-model-logging) o Neptune (https://neptune.ai/).


In [ ]:
folder + "/Model"

In [ ]:
# Usa el siguiente codigo para correr la funcion de entrenamiento

history= baseline.train(
    train_df=train_df,       # Set de entrenamiento
    validation_df=valid_df,  # Set de validacion
    save_path= folder + "/Model", # Ruta donde los modelos van a ser guardados
    epochs= 10,            # Numero de epocs
    batch_size= 32,        # Tamaño de lote
    #save_interval=20,     # Guarda un nuevo modelo canda X epochs (El mejor modelos siempre se guarda)
    num_workers=0
)
plot_scatter(baseline,"Baseline")

baseline_test = baseline.predict(testing_list)
plot_testing_hist(baseline_test,testing_df,"Baseline model separability")


# 3.1. Datos y preprocesamiento
# 3.1.1. Dataset

El tamaño del conjunto de entrenamiento importa mucho; un clasificador no puede aprender una variación que nunca se le ha mostrado. Además, en muchos casos hay poca variación entre clases.


# 3.1.1.1. Balance de dataset



#  Entrena el modelo con un set de entrenamiento desbalanceado

In [ ]:

# Remueve la mitad de los cantos positivos (clase BOANA)
indices = label[label['boana'] == 1].index
np.random.seed(42)
label_unbalanced = label.drop(np.random.choice(indices, size=len(indices) // 2, replace=False))

# Divide el dataset mientras entrenamiento y evaluacion
train_set, valid_set = train_test_split(label_unbalanced, test_size=0.2,random_state=0)

print(f"El numero de cantos en el set balanceado es:")
print(label['boana'].value_counts())
print()
print(f"El numero de cantos en el set desbalanceado es:")
print(label_unbalanced['boana'].value_counts())

In [ ]:
unbalanced = CNN('resnet18',classes=list(train_set.columns),sample_duration=3.0) # Create the CNN object

history = unbalanced.train(train_df=train_set, validation_df=valid_set,
                           save_path= folder + "/Model",
                           epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(baseline,"Baseline")


plot_scatter(unbalanced, title="Unbalanced model loss curve")

# Obten las predicciones y grafica los histogramas
unbalanced_test = unbalanced.predict(testing_list)
plot_testing_hist(unbalanced_test,testing_df," Unbalanced model separability")

# Remuestrea el sets de entrenamiento

In [ ]:
positives=200
negatives=100
index_pos=index_pos.head(positives)
index_neg=index_neg.head(negatives)
frames = [index_pos, index_neg]
label = pd.concat(frames)
label["boana"] = pd.to_numeric(label["boana"])

train_set, valid_set = train_test_split(label,test_size=0.2,random_state=0)

# Sobremuestreo (repetir muestras) para que todas las clases tengan x muestras
balanced_train_set = resample(train_df,n_samples_per_class=200,random_state=0)
print(f"Set de entrenamiento remuestreado con  {len(balanced_train_set)} cantos")

resampled = CNN('resnet18',classes=list(balanced_train_set.columns),sample_duration=3.0)

history = resampled.train(train_df=balanced_train_set, validation_df=valid_set,
                          save_path= folder +"/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(resampled, title="Resampled model loss curve")

# Obten las predicciones y grafica los histogramas
resampled_test = resampled.predict(testing_list)
plot_testing_hist(resampled_test,testing_df," Resampled model separability")



# 3.1.1.2. Origen de set de datos

# Modelo entrenado con set de datos subrepresentado geograficamente

La representatividad importa, la generalización del modelo se pondrá a prueba cuando ejecute su modelo en grabaciones de ARUs


In [ ]:
# Crear un nuevo conjunto de entrenamiento con grabaciones de una única ubicación (basado en Canas et al., 2023)

tar_p=folder+'/Training/BOAFAB_Location'
tar_n=folder+'/Training/OTHER'

In [ ]:
index_n=read_samples(tar_n,"0")
index_p=read_samples(tar_p,"1")

frames = [index_p, index_n]
tar_label = pd.concat(frames)

tar_label["boana"] = pd.to_numeric(tar_label["boana"])
tar_label

In [ ]:
train_set, valid_set = train_test_split(tar_label,test_size=0.2,random_state=0)
targeted = CNN('resnet18',classes=list(balanced_train_set.columns),sample_duration=3.0)


history = targeted.train(train_df=train_set, validation_df=valid_set,save_path= folder+"/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

# Obten las predicciones y grafica los histogramas
targeted_test = targeted.predict(testing_list)
plot_testing_hist(targeted_test,testing_df," Targeted model separability")

# 3.1.2. Preprocesamiento
# 3.1.2.1. Caracteristicas del espectrogramas
# Preprocesador con tamaño de ventana "incorrecto" (resolucion temporal vs espectral)


In [ ]:
from opensoundscape import AudioFileDataset, SpectrogramPreprocessor
from opensoundscape.preprocess.utils import show_tensor

spec_pre=SpectrogramPreprocessor(sample_duration=3)
dataset = AudioFileDataset(label,spec_pre)
dataset.bypass_augmentations=True

# Cambia el tamaño de ventana
window_size=1000 #@param

# Ejemplo
dataset.preprocessor.pipeline.to_spec.params.window_samples = window_size
show_tensor(dataset[0].data)

In [ ]:
# Hagamos la CNN y cambia el tamaño de ventana
window = CNN('resnet18',classes=list(train_set.columns),sample_duration=3.0)
window.preprocessor.pipeline.to_spec.params.window_samples = window_size
window.preprocessor.pipeline.to_spec.params

In [ ]:
# Carga el modelo de entrenamiento y evaluacion

history = window.train(train_df=train_set, validation_df=valid_set,save_path= folder + "/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(window, title="Window model loss curve")

# Obten las predicciones y grafica los histogramas
window_test = window.predict(testing_list)
plot_testing_hist(window_test,testing_df,"Window model separability")


# Preprocesamiento with filtro de bajo paso

In [ ]:
# Asi es como añadimos un filtro de banda
dataset.preprocessor.pipeline.bandpass.set(min_f=2000,max_f=4000)

print('Tensor incluyendo el filtro de banda')
show_tensor(dataset[0].data)

In [ ]:
# Añade filtro de banda
bandpass = CNN('resnet18',classes=list(train_set.columns),sample_duration=3.0) # Create the CNN
bandpass.preprocessor.pipeline.bandpass.set(min_f=2000,max_f=4000)
bandpass.preprocessor.pipeline.bandpass.params

In [ ]:
# Entrenamiento y evaluacion del modelo
history = bandpass.train(train_df=train_set, validation_df=valid_set,save_path= folder +"/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(bandpass, title="Bandpass model loss curve")

# Obten las predicciones y grafica los histogramas
bandpass_test = bandpass.predict(testing_list)
plot_testing_hist(bandpass_test,testing_df,"bandpass model separability")


# 3.1.2.2. Aumentacion de datos

Intenta incrementar la generalizacion del modelo evitando aumentacion de datos destructiva

# Preprocesador con aumentacion de datos

In [ ]:
# Añade la aumentacion de datos
augmentation = CNN('resnet18',classes=list(train_set.columns),sample_duration=3.0) # Create the CNN
augmentation.preprocessor.pipeline.bandpass.bypass = False
augmentation.preprocessor.pipeline.time_mask.bypass = False
augmentation.preprocessor.pipeline.frequency_mask.bypass = False
augmentation.preprocessor.pipeline.add_noise.bypass = False
augmentation.preprocessor.pipeline.random_affine.bypass = False
augmentation.preprocessor.pipeline

In [ ]:
# Entrenamiento y evaluacion del modelo

history = augmentation.train(train_df=train_set, validation_df=valid_set,save_path= folder +"/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(augmentation, title="Augmented model loss curve")

# Obten las predicciones y grafica los histogramas

augmentation_test = augmentation.predict(testing_list)
plot_testing_hist(augmentation_test,testing_df,"Augmented model separability")


# 3.2 Cambia los hyperparametros (Tasa de aprendizaje)

In [ ]:
# Modifica la tasa de aprendizaje
learning = CNN('resnet18',classes=list(train_set.columns),sample_duration=3.0) # Create the CNN
learning.optimizer_params['lr']=0.001
learning.optimizer_params

In [ ]:
# Entrenamiento y evaluacion del modelo

history = learning.train(train_df=train_set, validation_df=valid_set,save_path= folder +"/Model",
                       epochs= 10,batch_size= 32, num_workers=0)

plot_scatter(learning, title="Learning model loss curve")

# Obten las predicciones y grafica los histogramas

learning_test = learning.predict(testing_list)
plot_testing_hist(learning_test,testing_df,"Learning model separability")


# Evalua el modelo con cantos subrepresentados


In [ ]:
# Lista los archivos WAV en la carpeta de evaluacion
testing_list_loca=sorted(glob(folder+'/Testing_location/*.wav'))
# Final dataframe
testing_df_under = pd.DataFrame({'paths': testing_list_loca, 'labels': testing_labels})

under = load_model(folder + '/Model/best.model')

test_under = under.predict(testing_list_loca)
plot_testing_hist(test_under,testing_df_under,"Baseline model separability (underrepresented)")
